In [1]:
from pathlib import Path

import boto3

In [2]:
flag_is_ec2 = True

In [4]:
if flag_is_ec2 == True:
    #To access 's3' without any access key embedded following dependencies shall be met:
    # 1. Role : Allow-BAD-Project-S3-Passrole-to-EC2 
    # 2. Policy for user            : BearingAnomalyDetection_OperateS3Bucket

    aws_s3 = boto3.resource('s3')
    s3_bucket = aws_s3.Bucket('anomaly-detection-from-bearing-vibration-project-bucket')

    s3_bucket_objects=[]
    for s3_bucket_object in s3_bucket.objects.all():
        s3_bucket_objects.append(s3_bucket_object)

ClientError: An error occurred (AccessDenied) when calling the ListObjects operation: Access Denied

In [ ]:
s3_objects_1st_dataset=[]
s3_objects_2nd_dataset=[]
s3_objects_3rd_dataset=[]
paths = []

for s3_object in s3_bucket_objects:
    path_parts = Path(s3_object.key).parts
    if len(path_parts) == 4 and path_parts[0] == 'data_input' and path_parts[1] == 'IMS':
        paths.append(s3_object)
        if path_parts[2] == '1st_test':
            s3_objects_1st_dataset.append(s3_object)
        elif path_parts[2] == '2nd_test':
            s3_objects_2nd_dataset.append(s3_object)
        else:
            s3_objects_3rd_dataset.append(s3_object)

print('Number of files in 1st Dataset:', len(s3_objects_1st_dataset), 'first file=', s3_objects_1st_dataset[0].key)
print('Number of files in 2nd Dataset:', len(s3_objects_2nd_dataset), 'first file=', s3_objects_2nd_dataset[0].key)
print('Number of files in 3rd Dataset:', len(s3_objects_3rd_dataset), 'first file=', s3_objects_3rd_dataset[0].key)

In [ ]:
#1st Set has 8 
col_names_1st = ['b1_ch1', 'b1_ch2', 'b2_ch3', 'b2_ch4', 'b3_ch5', 'b3_ch6', 'b4_ch7', 'b4_ch8']
#2nd and 3rd has 4
col_names_2nd_3rd = ['b1_ch1', 'b2_ch2', 'b3_ch3', 'b4_ch4']

data_set_paths=[]
data_set_paths= [s3_objects_1st_dataset, s3_objects_2nd_dataset, s3_objects_3rd_dataset]

In [ ]:
from io import BytesIO
import pandas as pd

df = []
for s3_object in s3_objects_1st_dataset[:1]:
    data = s3_object.get()['Body'].read()
    df.append(pd.read_csv(BytesIO(data), header=None, delimiter='\t', low_memory='False'))


In [ ]:
df[0]